# Assessing greenspace accessability within Northern Ireland through an interactive map which calculates the nearest proximity of a settlement to its nearest off-road greensapce trial. 
## Overview 
In this code, you will see how you can use libaries like geopandas and folium in-conjunction with functions like STRtree and nearest_points, to understand how varied greenspace accessability really is within Northern Ireland. Whilst there are more accurate models of greenspace accessability within Northern Ireland, the importance of how accessable a greenspace is goes unnoticed in todays society and although they have great social and cultural values, greenspaces are not accessabile to everyone and therefore, this map shows a robust approach on how you can analyse greensapce accessability. 

## Objectives 
1. Use `geopandas` to __load__ and __re-project__ data files into their correct __EPSG__
2. Use `geodataframe` functions to __store__ settlement data as __centroids__, produce __connection lines__ between the nearest off-road trail and the nearest settlement and finally to __calculate__ the areas of the greenspace polygons
3. Use `shapely` tools including _STRtree_, _nearest_points_ and _LineString_ to calculate the __Euclidean__ distance from each settlement to their closest off-road trail
4. Use `folium` to create an __interactive__ web based map that includes various features like __popups__, __hovering__ over features and a __layer control panel__

## Time to get started

In [ ]:
""" 
Imports libaries and tools that are required to conduct the analysis. 
"""
import geopandas as gpd
import folium
from shapely.geometry import LineString
from shapely.strtree import STRtree
from shapely.ops import nearest_points

In [ ]:
"""
Loads settlement, trail, and greenspace shapefiles.
"""
settlements = gpd.read_file("data/Settlements/settlements-2015-above-500-threshold.shp")
trails = gpd.read_file("data/Trails/greenspaceoffroadtrails.shp")
greenspace = gpd.read_file("data/Greenspace/greenspace.shp")

In [ ]:
"""
Checks the EPSG for the settlements data file.
"""
settlements.crs

In [ ]:
"""
Checks the EPSG for the trails data file.
"""
trails.crs

In [ ]:
"""
Checks the EPSG for the greenspace data file.
"""
greenspace.crs

In [ ]:
"""
Reprojects all spatial layers to EPSG:3857, to ensure that area is calculated in the correct units. 
"""
settlements = settlements.to_crs(epsg=3857)
trails = trails.to_crs(epsg=3857)
greenspace = greenspace.to_crs(epsg=3857)

In [ ]:
"""
Re-checks the EPSG for the settlements data file.
"""
settlements.crs

In [ ]:
"""
Re-checks the EPSG for the trails data file.
"""
trails.crs

In [ ]:
"""
Re-checks the EPSG for the greenspace data file.
"""
greenspace.crs

In [ ]:
"""
Displays the first 5 rows of the greenspace shapefile.
"""
greenspace.head(5)

In [ ]:
"""
Displays the first 5 rows of the settlements shapefile.
"""
settlements.head(5)

In [ ]:
"""
Displays the first 5 rows of the trails shapefile.
"""
trails.head(5)

In [ ]:
"""
Rectifies invalid feature geometries to avoid errors during the distance calculations, spatial indexing when using STRtree and nearest_points and finally, to ensure the map is rendered smoothly.
"""

# Code for validating features is adapted from: https://app.readthedocs.org/projects/shapely/downloads/pdf/latest/
settlements = settlements[settlements.is_valid]
trails = trails[trails.is_valid]
greenspace = greenspace[greenspace.is_valid]

In [ ]:
"""
Calculates area, converts it into hectares for each greenspace polygon and stores its data as centroids.
"""
if 'Area_Ha' not in greenspace.columns:
    greenspace['Area_Ha'] = greenspace.geometry.area / 10_000  # Convert m² to ha for an accurate representation of area

greenspace['centroid'] = greenspace.geometry.centroid # Adds a centriod column to the greenspace geo data frame

In [ ]:
"""
Uses a spatial index through the STRtree and nearest_points functions to calculate and store the nearest off-road trial distance to the closest settlements. LineStrings is then used to visualise the distance between the two features 
"""

# Code implementing STRtree and nearest_points functions is from: https://shapely.readthedocs.io/en/stable/strtree.html#r77d4c1f42cb7-1
trail_geoms = trails.geometry.tolist() # Converts to a list of individual geometries
trail_tree = STRtree(trail_geoms) # Generates a spatial index 

connection_lines = []  # Creates an empty list to hold LineStrings from settlements to off-road trails
distances = []         # Creates an empty list to hold the calculated Euclidean distances in m

for idx, row in settlements.iterrows(): # Loops through each settlement to process the following details:
    point = row.geometry.centroid # Finds the center point of the settlements centriod 
    nearest_idx = trail_tree.nearest(point) # Finds which off-road trails are the closest to each settlement 
    nearest_geom = trail_geoms[nearest_idx] # Finds the geometry of those nearest off-road trials 
    nearest_point = nearest_points(point, nearest_geom)[1] # Finds the point on the off-road trail that is closest to the point on the settlement  

    line = LineString([point, nearest_point]) # Connects the nearest off-road trail point to the nearest settlement point
    distance = point.distance(nearest_point) # Measures the distance from the nearest off-road trails to the nearest settlement

    connection_lines.append(line) # Stores the lines that connects the settlements to the off-road trails
    distances.append(distance) # Stores the distance between the settlements and off-road trails 

connections = gpd.GeoDataFrame(
    {'distance_m': distances}, # Adds the calculated distances as a new attribute called 'distance_m'
    geometry=connection_lines, # Adds the calculates lines to the new attribute
    crs=settlements.crs # Uses the same CRS as settlements data file
)

In [ ]:
"""
Checks the EPSG of the connections file.
"""
connections.crs

In [ ]:
"""
Reprojects all spatial layers to EPSG:4326 to allow folium to operate.
"""
settlements = settlements.to_crs(epsg=4326)
trails = trails.to_crs(epsg=4326)
greenspace = greenspace.to_crs(epsg=4326)
connections = connections.to_crs(epsg=4326)

In [ ]:
"""
Re-checks the EPSG of the settlements data file.
"""
settlements.crs

In [ ]:
"""
Re-checks the EPSG of the trails data file.
"""
trails.crs

In [ ]:
"""
Re-checks the EPSG of the greenspace data file.
"""
greenspace.crs

In [ ]:
"""
Re-checks the EPSG of the connections data file.
"""
connections.crs

In [ ]:
"""
Creates a folium web map that centers the map to the centroids of all the settlement points.
"""
center = settlements.geometry.union_all().centroid # Merges the settlements before finding its centroid 
m = folium.Map(location=[center.y, center.x], zoom_start=8) # Sets the web map to open showing the center points of all settlements at a zoom of 8


In [ ]:
"""
Adds greenspace polygons to the map with popups revealing its name, category and area.
"""

# Code for lambda styling adapted from: https://medium.com/%40aakash013/geospatial-data-visualization-with-folium-geopandas-092673eeaa35  
for _, row in greenspace.iterrows(): # Loops through each greenspace to process the following details:
    popup = folium.Popup(
        f"<strong>Name:</strong> {row.get('Name', 'N/A')}<br>" # Displays the name of each greenspace 
        f"<strong>Category:</strong> {row.get('Category','N/A')}<br>" # Displays the category of each greenspace 
        f"<strong>Area:</strong> {round(row['Area_Ha'], 2)} ha", # Displays the area of each greenspace 
        max_width=250
    )
    folium.GeoJson(
        row.geometry,
        style_function=lambda x: { # Style used 
            'fillColor': 'darkorange', # Fill colour
            'color': 'darkorange', # Border colour 
            'fillOpacity': 0.4, # Transparency of the polygon 
            'weight': 0.5 # Thickness of border 
        },
        popup=popup # Attaches the specified characteristics to the map
    ).add_to(m) # Adds the style to the map

In [ ]:
"""
Adds off-road trail lines to the map that are clickable and show the trail names.
"""
folium.GeoJson(
    trails, # The file that contains the off-road trail entities 
    name='Trails', # The name of the layer in the layer control panel 
    style_function=lambda x: { # The visual style used  
        'color': 'blue', # Off-road trail Colour 
        'weight': 2 # Thickness of off-road trail feature 
    },
    tooltip=folium.GeoJsonTooltip( # Allows the feature to be hovered over 
        fields=['Name'], # Displays the name of the off-road trails            
        aliases=['Trail Name:'], # How the 'Name' is labelled 
        localize=True,
        sticky=True
    )
).add_to(m) # Adds the style to the map

In [ ]:
"""
Adds settlements to the map as black circles
"""
for _, row in settlements.iterrows(): # Loops through each settlement to process the following details:
    centroid = row.geometry.centroid # Uses the center of the settlements 
    folium.CircleMarker(
        location=[centroid.y, centroid.x], # Utalises the coordinates of the centroids
        radius=4, # Size of the centroid
        color='black', # Colour of the centroid
        fill=True,
        fill_opacity=0.7, # Transparency of the centroid
        tooltip=row.get("Name", "N/A") # Displays the name of each settlement 
    ).add_to(m) # Adds the style to the map

In [ ]:
"""
Draws red lines from each settlement to the nearest off-road trail. Also allows off-road trail lines to be toggled on/off and when hovered over, it will reveal the calculated distances.
"""
folium.GeoJson(
    connections, # The file that contains the connections entities
    name='Nearest Trail Lines', # The name of the layer in the layer control panel 
    style_function=lambda x: { # The visual style used  
        'color': 'red', # Connection lines colour
        'weight': 1.5 # Thickness of the connections feature 
    },
    tooltip=folium.GeoJsonTooltip( # Allows the feature to be hovered over 
        fields=['distance_m'], # Displays the calculated distance of the closest off-road trial to the closest settlement 
        aliases=['Distance (m):'], # How the 'Name' is labelled 
        localize=True
    )
).add_to(m) # Adds the style to the map   

In [ ]:
"""
Adds a layer control panel to the web page and exports the map as a HTML file. 
"""

# Code for layer control adapted from: https://python-visualization.github.io/folium/latest/user_guide/ui_elements/layer_control.html
folium.LayerControl().add_to(m) # Enables the layer control panel
m.save("nearest_trail_access_map.html") # Creates the interactive web page 

print('nearest_trail_access_map.html') # Creates a saved file as an output 

## Conclusion 
I hope that this interactive web based map allows you to see how accessable a greenspace really is to you. If you get stuck, remember to use the tips in the how-to-guide and I hope that you have taken something away from this code. Thank you!!